In [77]:
import pandas as pd
%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.signal import find_peaks


In [78]:


df0 = pd.read_csv("/Users/kiranbhaskar/Desktop/Supercomputing/activityrecognition.csv")
df0.columns = range(df0.columns.size)
df0 = df0.rename(columns = {0:'user',1:'activity',2:'timestamp',3:'x-axis',4:'y-axis',5:'z-axis'})
df0 = df0.iloc[:, :-5]

/Users/kiranbhaskar/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (6,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [79]:
har_df = df0
# removing null values
har_df = har_df.dropna()
har_df.shape

# transforming the z-axis to float
har_df['z-axis'] = har_df['z-axis'].str.replace(';', '')
har_df['z-axis'] = har_df['z-axis'].apply(lambda x:float(x))

# drop the rows where timestamp is 0
df = har_df[har_df['timestamp'] != 0]

# now arrange data in ascending order of the user and timestamp
df = df.sort_values(by = ['user', 'timestamp'], ignore_index=True)

<ipython-input-79-157af39b973a>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  har_df['z-axis'] = har_df['z-axis'].str.replace(';', '')
<ipython-input-79-157af39b973a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  har_df['z-axis'] = har_df['z-axis'].apply(lambda x:float(x))


In [121]:

x_list = []
y_list = []
z_list = []
train_labels = []

window_size = 16
step_size = 16
fft = 8
# creating overlaping windows of size window-size 100
for i in range(0, df.shape[0] - window_size, step_size):
    xs = df['x-axis'].values[i: i + window_size]
    ys = df['y-axis'].values[i: i + window_size]
    zs = df['z-axis'].values[i: i + window_size]
    label = stats.mode(df['activity'][i: i + window_size])[0][0]

    x_list.append(xs)
    y_list.append(ys)
    z_list.append(zs)
    train_labels.append(label)

# Statistical Features on raw x, y and z in time domain
X_train = pd.DataFrame()

# mean
X_train['x_mean'] = pd.Series(x_list).apply(lambda x: x.mean())
X_train['y_mean'] = pd.Series(y_list).apply(lambda x: x.mean())
X_train['z_mean'] = pd.Series(z_list).apply(lambda x: x.mean())

# std dev
X_train['x_std'] = pd.Series(x_list).apply(lambda x: x.std())
X_train['y_std'] = pd.Series(y_list).apply(lambda x: x.std())
X_train['z_std'] = pd.Series(z_list).apply(lambda x: x.std())

# avg absolute diff
X_train['x_aad'] = pd.Series(x_list).apply(lambda x: np.mean(np.absolute(x - np.mean(x))))
X_train['y_aad'] = pd.Series(y_list).apply(lambda x: np.mean(np.absolute(x - np.mean(x))))
X_train['z_aad'] = pd.Series(z_list).apply(lambda x: np.mean(np.absolute(x - np.mean(x))))

# min
X_train['x_min'] = pd.Series(x_list).apply(lambda x: x.min())
X_train['y_min'] = pd.Series(y_list).apply(lambda x: x.min())
X_train['z_min'] = pd.Series(z_list).apply(lambda x: x.min())

# max
X_train['x_max'] = pd.Series(x_list).apply(lambda x: x.max())
X_train['y_max'] = pd.Series(y_list).apply(lambda x: x.max())
X_train['z_max'] = pd.Series(z_list).apply(lambda x: x.max())

# max-min diff
X_train['x_maxmin_diff'] = X_train['x_max'] - X_train['x_min']
X_train['y_maxmin_diff'] = X_train['y_max'] - X_train['y_min']
X_train['z_maxmin_diff'] = X_train['z_max'] - X_train['z_min']

# median
X_train['x_median'] = pd.Series(x_list).apply(lambda x: np.median(x))
X_train['y_median'] = pd.Series(y_list).apply(lambda x: np.median(x))
X_train['z_median'] = pd.Series(z_list).apply(lambda x: np.median(x))

# median abs dev 
X_train['x_mad'] = pd.Series(x_list).apply(lambda x: np.median(np.absolute(x - np.median(x))))
X_train['y_mad'] = pd.Series(y_list).apply(lambda x: np.median(np.absolute(x - np.median(x))))
X_train['z_mad'] = pd.Series(z_list).apply(lambda x: np.median(np.absolute(x - np.median(x))))

# interquartile range
X_train['x_IQR'] = pd.Series(x_list).apply(lambda x: np.percentile(x, 75) - np.percentile(x, 25))
X_train['y_IQR'] = pd.Series(y_list).apply(lambda x: np.percentile(x, 75) - np.percentile(x, 25))
X_train['z_IQR'] = pd.Series(z_list).apply(lambda x: np.percentile(x, 75) - np.percentile(x, 25))

# negtive count
X_train['x_neg_count'] = pd.Series(x_list).apply(lambda x: np.sum(x < 0))
X_train['y_neg_count'] = pd.Series(y_list).apply(lambda x: np.sum(x < 0))
X_train['z_neg_count'] = pd.Series(z_list).apply(lambda x: np.sum(x < 0))

# positive count
X_train['x_pos_count'] = pd.Series(x_list).apply(lambda x: np.sum(x > 0))
X_train['y_pos_count'] = pd.Series(y_list).apply(lambda x: np.sum(x > 0))
X_train['z_pos_count'] = pd.Series(z_list).apply(lambda x: np.sum(x > 0))

# values above mean
X_train['x_above_mean'] = pd.Series(x_list).apply(lambda x: np.sum(x > x.mean()))
X_train['y_above_mean'] = pd.Series(y_list).apply(lambda x: np.sum(x > x.mean()))
X_train['z_above_mean'] = pd.Series(z_list).apply(lambda x: np.sum(x > x.mean()))

# number of peaks
X_train['x_peak_count'] = pd.Series(x_list).apply(lambda x: len(find_peaks(x)[0]))
X_train['y_peak_count'] = pd.Series(y_list).apply(lambda x: len(find_peaks(x)[0]))
X_train['z_peak_count'] = pd.Series(z_list).apply(lambda x: len(find_peaks(x)[0]))

# skewness
X_train['x_skewness'] = pd.Series(x_list).apply(lambda x: stats.skew(x))
X_train['y_skewness'] = pd.Series(y_list).apply(lambda x: stats.skew(x))
X_train['z_skewness'] = pd.Series(z_list).apply(lambda x: stats.skew(x))

# kurtosis
X_train['x_kurtosis'] = pd.Series(x_list).apply(lambda x: stats.kurtosis(x))
X_train['y_kurtosis'] = pd.Series(y_list).apply(lambda x: stats.kurtosis(x))
X_train['z_kurtosis'] = pd.Series(z_list).apply(lambda x: stats.kurtosis(x))

# energy
X_train['x_energy'] = pd.Series(x_list).apply(lambda x: np.sum(x**2)/window_size)
X_train['y_energy'] = pd.Series(y_list).apply(lambda x: np.sum(x**2)/window_size)
X_train['z_energy'] = pd.Series(z_list).apply(lambda x: np.sum(x**2/window_size))

# avg resultant
X_train['avg_result_accl'] = [i.mean() for i in ((pd.Series(x_list)**2 + pd.Series(y_list)**2 + pd.Series(z_list)**2)**0.5)]

# signal magnitude area
X_train['sma'] =    pd.Series(x_list).apply(lambda x: np.sum(abs(x)/window_size)) + pd.Series(y_list).apply(lambda x: np.sum(abs(x)/window_size)) \
                  + pd.Series(z_list).apply(lambda x: np.sum(abs(x)/window_size))
# converting the signals from time domain to frequency domain using FFT
x_list_fft = pd.Series(x_list).apply(lambda x: np.abs(np.fft.fft(x))[1:fft])
y_list_fft = pd.Series(y_list).apply(lambda x: np.abs(np.fft.fft(x))[1:fft])
z_list_fft = pd.Series(z_list).apply(lambda x: np.abs(np.fft.fft(x))[1:fft])

# Statistical Features on raw x, y and z in frequency domain
# FFT mean
X_train['x_mean_fft'] = pd.Series(x_list_fft).apply(lambda x: x.mean())
X_train['y_mean_fft'] = pd.Series(y_list_fft).apply(lambda x: x.mean())
X_train['z_mean_fft'] = pd.Series(z_list_fft).apply(lambda x: x.mean())

# FFT std dev
X_train['x_std_fft'] = pd.Series(x_list_fft).apply(lambda x: x.std())
X_train['y_std_fft'] = pd.Series(y_list_fft).apply(lambda x: x.std())
X_train['z_std_fft'] = pd.Series(z_list_fft).apply(lambda x: x.std())

# FFT avg absolute diff
X_train['x_aad_fft'] = pd.Series(x_list_fft).apply(lambda x: np.mean(np.absolute(x - np.mean(x))))
X_train['y_aad_fft'] = pd.Series(y_list_fft).apply(lambda x: np.mean(np.absolute(x - np.mean(x))))
X_train['z_aad_fft'] = pd.Series(z_list_fft).apply(lambda x: np.mean(np.absolute(x - np.mean(x))))

# FFT min
X_train['x_min_fft'] = pd.Series(x_list_fft).apply(lambda x: x.min())
X_train['y_min_fft'] = pd.Series(y_list_fft).apply(lambda x: x.min())
X_train['z_min_fft'] = pd.Series(z_list_fft).apply(lambda x: x.min())

# FFT max
X_train['x_max_fft'] = pd.Series(x_list_fft).apply(lambda x: x.max())
X_train['y_max_fft'] = pd.Series(y_list_fft).apply(lambda x: x.max())
X_train['z_max_fft'] = pd.Series(z_list_fft).apply(lambda x: x.max())

# FFT max-min diff
X_train['x_maxmin_diff_fft'] = X_train['x_max_fft'] - X_train['x_min_fft']
X_train['y_maxmin_diff_fft'] = X_train['y_max_fft'] - X_train['y_min_fft']
X_train['z_maxmin_diff_fft'] = X_train['z_max_fft'] - X_train['z_min_fft']

# FFT median
X_train['x_median_fft'] = pd.Series(x_list_fft).apply(lambda x: np.median(x))
X_train['y_median_fft'] = pd.Series(y_list_fft).apply(lambda x: np.median(x))
X_train['z_median_fft'] = pd.Series(z_list_fft).apply(lambda x: np.median(x))

# FFT median abs dev 
X_train['x_mad_fft'] = pd.Series(x_list_fft).apply(lambda x: np.median(np.absolute(x - np.median(x))))
X_train['y_mad_fft'] = pd.Series(y_list_fft).apply(lambda x: np.median(np.absolute(x - np.median(x))))
X_train['z_mad_fft'] = pd.Series(z_list_fft).apply(lambda x: np.median(np.absolute(x - np.median(x))))

# FFT Interquartile range
X_train['x_IQR_fft'] = pd.Series(x_list_fft).apply(lambda x: np.percentile(x, 75) - np.percentile(x, 25))
X_train['y_IQR_fft'] = pd.Series(y_list_fft).apply(lambda x: np.percentile(x, 75) - np.percentile(x, 25))
X_train['z_IQR_fft'] = pd.Series(z_list_fft).apply(lambda x: np.percentile(x, 75) - np.percentile(x, 25))

# FFT values above mean
X_train['x_above_mean_fft'] = pd.Series(x_list_fft).apply(lambda x: np.sum(x > x.mean()))
X_train['y_above_mean_fft'] = pd.Series(y_list_fft).apply(lambda x: np.sum(x > x.mean()))
X_train['z_above_mean_fft'] = pd.Series(z_list_fft).apply(lambda x: np.sum(x > x.mean()))

# FFT number of peaks
X_train['x_peak_count_fft'] = pd.Series(x_list_fft).apply(lambda x: len(find_peaks(x)[0]))
X_train['y_peak_count_fft'] = pd.Series(y_list_fft).apply(lambda x: len(find_peaks(x)[0]))
X_train['z_peak_count_fft'] = pd.Series(z_list_fft).apply(lambda x: len(find_peaks(x)[0]))

# FFT skewness
X_train['x_skewness_fft'] = pd.Series(x_list_fft).apply(lambda x: stats.skew(x))
X_train['y_skewness_fft'] = pd.Series(y_list_fft).apply(lambda x: stats.skew(x))
X_train['z_skewness_fft'] = pd.Series(z_list_fft).apply(lambda x: stats.skew(x))

# FFT kurtosis
X_train['x_kurtosis_fft'] = pd.Series(x_list_fft).apply(lambda x: stats.kurtosis(x))
X_train['y_kurtosis_fft'] = pd.Series(y_list_fft).apply(lambda x: stats.kurtosis(x))
X_train['z_kurtosis_fft'] = pd.Series(z_list_fft).apply(lambda x: stats.kurtosis(x))

# FFT energy
X_train['x_energy_fft'] = pd.Series(x_list_fft).apply(lambda x: np.sum(x**2)/((window_size)/2))
X_train['y_energy_fft'] = pd.Series(y_list_fft).apply(lambda x: np.sum(x**2)/((window_size)/2))
X_train['z_energy_fft'] = pd.Series(z_list_fft).apply(lambda x: np.sum(x**2/((window_size)/2)))

# FFT avg resultant
X_train['avg_result_accl_fft'] = [i.mean() for i in ((pd.Series(x_list_fft)**2 + pd.Series(y_list_fft)**2 + pd.Series(z_list_fft)**2)**0.5)]

# FFT Signal magnitude area
X_train['sma_fft'] = pd.Series(x_list_fft).apply(lambda x: np.sum(abs(x)/((window_size)/2))) + pd.Series(y_list_fft).apply(lambda x: np.sum(abs(x)/((window_size)/2))) \
                     + pd.Series(z_list_fft).apply(lambda x: np.sum(abs(x)/((window_size)/2)))
# Max Indices and Min indices 

# index of max value in time domain
X_train['x_argmax'] = pd.Series(x_list).apply(lambda x: np.argmax(x))
X_train['y_argmax'] = pd.Series(y_list).apply(lambda x: np.argmax(x))
X_train['z_argmax'] = pd.Series(z_list).apply(lambda x: np.argmax(x))

# index of min value in time domain
X_train['x_argmin'] = pd.Series(x_list).apply(lambda x: np.argmin(x))
X_train['y_argmin'] = pd.Series(y_list).apply(lambda x: np.argmin(x))
X_train['z_argmin'] = pd.Series(z_list).apply(lambda x: np.argmin(x))

# absolute difference between above indices
X_train['x_arg_diff'] = abs(X_train['x_argmax'] - X_train['x_argmin'])
X_train['y_arg_diff'] = abs(X_train['y_argmax'] - X_train['y_argmin'])
X_train['z_arg_diff'] = abs(X_train['z_argmax'] - X_train['z_argmin'])

# index of max value in frequency domain
X_train['x_argmax_fft'] = pd.Series(x_list_fft).apply(lambda x: np.argmax(np.abs(np.fft.fft(x))[1:fft]))
X_train['y_argmax_fft'] = pd.Series(y_list_fft).apply(lambda x: np.argmax(np.abs(np.fft.fft(x))[1:fft]))
X_train['z_argmax_fft'] = pd.Series(z_list_fft).apply(lambda x: np.argmax(np.abs(np.fft.fft(x))[1:fft]))

# index of min value in frequency domain
X_train['x_argmin_fft'] = pd.Series(x_list_fft).apply(lambda x: np.argmin(np.abs(np.fft.fft(x))[1:fft]))
X_train['y_argmin_fft'] = pd.Series(y_list_fft).apply(lambda x: np.argmin(np.abs(np.fft.fft(x))[1:fft]))
X_train['z_argmin_fft'] = pd.Series(z_list_fft).apply(lambda x: np.argmin(np.abs(np.fft.fft(x))[1:fft]))

# absolute difference between above indices
X_train['x_arg_diff_fft'] = abs(X_train['x_argmax_fft'] - X_train['x_argmin_fft'])
X_train['y_arg_diff_fft'] = abs(X_train['y_argmax_fft'] - X_train['y_argmin_fft'])
X_train['z_arg_diff_fft'] = abs(X_train['z_argmax_fft'] - X_train['z_argmin_fft'])



from scipy import signal

# Assume data is a list of lists, where each inner list contains 66 Fourier frequencies

# Calculate the PSD of each sample
x_list_psd = []
y_list_psd = []
z_list_psd = []
for sample in x_list_fft:
    _, psd = signal.periodogram(sample)
    x_list_psd.append(psd)
for sample in y_list_fft:
    _, psd = signal.periodogram(sample)
    y_list_psd.append(psd)
for sample in z_list_fft:
    _, psd = signal.periodogram(sample)
    z_list_psd.append(psd)
# psds will be a list of numpy arrays, where each array corresponds to a sample

# Statistical Features on raw x, y and z in frequency domain
# FFT mean
X_train['x_mean_psd'] = pd.Series(x_list_psd).apply(lambda x: x.mean())
X_train['y_mean_psd'] = pd.Series(y_list_psd).apply(lambda x: x.mean())
X_train['z_mean_psd'] = pd.Series(z_list_psd).apply(lambda x: x.mean())



# psd std dev
X_train['x_std_psd'] = pd.Series(x_list_psd).apply(lambda x: x.std())
X_train['y_std_psd'] = pd.Series(y_list_psd).apply(lambda x: x.std())
X_train['z_std_psd'] = pd.Series(z_list_psd).apply(lambda x: x.std())

# psd avg absolute diff
X_train['x_aad_psd'] = pd.Series(x_list_psd).apply(lambda x: np.mean(np.absolute(x - np.mean(x))))
X_train['y_aad_psd'] = pd.Series(y_list_psd).apply(lambda x: np.mean(np.absolute(x - np.mean(x))))
X_train['z_aad_psd'] = pd.Series(z_list_psd).apply(lambda x: np.mean(np.absolute(x - np.mean(x))))

# psd min
X_train['x_min_psd'] = pd.Series(x_list_psd).apply(lambda x: x.min())
X_train['y_min_psd'] = pd.Series(y_list_psd).apply(lambda x: x.min())
X_train['z_min_psd'] = pd.Series(z_list_psd).apply(lambda x: x.min())

# psd max
X_train['x_max_psd'] = pd.Series(x_list_psd).apply(lambda x: x.max())
X_train['y_max_psd'] = pd.Series(y_list_psd).apply(lambda x: x.max())
X_train['z_max_psd'] = pd.Series(z_list_psd).apply(lambda x: x.max())

# psd max-min diff
X_train['x_maxmin_diff_psd'] = X_train['x_max_psd'] - X_train['x_min_psd']
X_train['y_maxmin_diff_psd'] = X_train['y_max_psd'] - X_train['y_min_psd']
X_train['z_maxmin_diff_psd'] = X_train['z_max_psd'] - X_train['z_min_psd']

# psd median
X_train['x_median_psd'] = pd.Series(x_list_psd).apply(lambda x: np.median(x))
X_train['y_median_psd'] = pd.Series(y_list_psd).apply(lambda x: np.median(x))
X_train['z_median_psd'] = pd.Series(z_list_psd).apply(lambda x: np.median(x))

# psd median abs dev 
X_train['x_mad_psd'] = pd.Series(x_list_psd).apply(lambda x: np.median(np.absolute(x - np.median(x))))
X_train['y_mad_psd'] = pd.Series(y_list_psd).apply(lambda x: np.median(np.absolute(x - np.median(x))))
X_train['z_mad_psd'] = pd.Series(z_list_psd).apply(lambda x: np.median(np.absolute(x - np.median(x))))

# psd Interquartile range
X_train['x_IQR_psd'] = pd.Series(x_list_psd).apply(lambda x: np.percentile(x, 75) - np.percentile(x, 25))
X_train['y_IQR_psd'] = pd.Series(y_list_psd).apply(lambda x: np.percentile(x, 75) - np.percentile(x, 25))
X_train['z_IQR_psd'] = pd.Series(z_list_psd).apply(lambda x: np.percentile(x, 75) - np.percentile(x, 25))

# psd values above mean
X_train['x_above_mean_psd'] = pd.Series(x_list_psd).apply(lambda x: np.sum(x > x.mean()))
X_train['y_above_mean_psd'] = pd.Series(y_list_psd).apply(lambda x: np.sum(x > x.mean()))
X_train['z_above_mean_psd'] = pd.Series(z_list_psd).apply(lambda x: np.sum(x > x.mean()))

# psd number of peaks
X_train['x_peak_count_psd'] = pd.Series(x_list_psd).apply(lambda x: len(find_peaks(x)[0]))
X_train['y_peak_count_psd'] = pd.Series(y_list_psd).apply(lambda x: len(find_peaks(x)[0]))
X_train['z_peak_count_psd'] = pd.Series(z_list_psd).apply(lambda x: len(find_peaks(x)[0]))

# psd skewness
X_train['x_skewness_psd'] = pd.Series(x_list_psd).apply(lambda x: stats.skew(x))
X_train['y_skewness_psd'] = pd.Series(y_list_psd).apply(lambda x: stats.skew(x))
X_train['z_skewness_psd'] = pd.Series(z_list_psd).apply(lambda x: stats.skew(x))

# psd kurtosis
X_train['x_kurtosis_psd'] = pd.Series(x_list_psd).apply(lambda x: stats.kurtosis(x))
X_train['y_kurtosis_psd'] = pd.Series(y_list_psd).apply(lambda x: stats.kurtosis(x))
X_train['z_kurtosis_psd'] = pd.Series(z_list_psd).apply(lambda x: stats.kurtosis(x))

# psd energy
X_train['x_energy_psd'] = pd.Series(x_list_psd).apply(lambda x: np.sum(x**2)/((window_size)/2))
X_train['y_energy_psd'] = pd.Series(y_list_psd).apply(lambda x: np.sum(x**2)/((window_size)/2))
X_train['z_energy_psd'] = pd.Series(z_list_psd).apply(lambda x: np.sum(x**2/((window_size)/2)))

# psd avg resultant
X_train['avg_result_accl_psd'] = [i.mean() for i in ((pd.Series(x_list_psd)**2 + pd.Series(y_list_psd)**2 + pd.Series(z_list_psd)**2)**0.5)]

# psd Signal magnitude area
X_train['sma_psd'] = pd.Series(x_list_psd).apply(lambda x: np.sum(abs(x)/((window_size)/2))) + pd.Series(y_list_psd).apply(lambda x: np.sum(abs(x)/((window_size)/2))) \
                     + pd.Series(z_list_psd).apply(lambda x: np.sum(abs(x)/((window_size)/2)))

In [122]:
X_train

,x_mean,y_mean,z_mean,x_std,y_std,z_std,x_aad,y_aad,z_aad,x_min,...,y_skewness_psd,z_skewness_psd,x_kurtosis_psd,y_kurtosis_psd,z_kurtosis_psd,x_energy_psd,y_energy_psd,z_energy_psd,avg_result_accl_psd,sma_psd
0,3.126875,8.860000,-0.400625,5.386426,2.804561,2.996126,3.961484,2.246250,1.822734,-4.59,...,-0.162185,0.244272,-0.732405,-1.716879,-1.318489,87062.106672,1123.343724,53.750228,253.909929,147.051217
1,3.613750,10.458125,-0.356875,6.144270,4.584745,2.818317,5.180469,3.910859,2.281406,-5.13,...,1.054460,0.398994,-0.743875,-0.755273,-1.434580,53049.548471,13545.049533,6714.893586,246.462713,185.310812
2,3.700000,9.505625,-0.312500,4.333953,3.777830,3.703416,3.108750,3.102422,2.680313,-6.66,...,1.108760,1.145072,-1.821158,-0.700813,-0.673742,2072.629022,58072.666999,3212.864838,233.091487,147.132876
3,4.718125,11.327500,-0.672500,5.280230,3.239933,3.586068,4.086641,2.394062,2.620625,-3.17,...,-0.094048,0.757128,-1.167529,-0.998302,-0.932999,13350.334066,17609.953460,1559.304879,210.541858,163.754631
4,3.058750,8.770000,-0.210000,5.770532,3.441110,3.822005,4.453750,3.038750,2.758750,-7.35,...,1.008154,0.909023,-0.882539,-0.791022,-0.901228,98389.948572,17011.050595,13509.406360,354.429583,265.664085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64727,-1.285000,9.451250,2.447406,0.056899,0.044843,0.047846,0.042500,0.036094,0.037030,-1.33,...,0.959301,0.207399,-0.914876,-0.789138,-1.082370,0.000344,0.000208,0.000035,0.025079,0.019365
64728,-1.371875,9.430625,2.472093,0.066729,0.044366,0.045684,0.054844,0.029609,0.038307,-1.53,...,0.031136,0.719525,-1.617929,-1.894509,-1.086349,0.000031,0.000239,0.000242,0.025151,0.018799
64729,-1.381875,9.419375,2.429530,0.081180,0.070309,0.048365,0.071406,0.059375,0.039797,-1.53,...,-0.076642,0.782299,-0.729114,-1.115538,-0.932255,0.005417,0.001781,0.000191,0.086211,0.061506
64730,-1.380625,9.460625,2.398032,0.068872,0.052734,0.034547,0.059531,0.042891,0.028092,-1.50,...,0.179169,0.175450,-1.238162,-1.733629,-1.765349,0.001708,0.000099,0.000001,0.049629,0.029695


In [123]:
X_train['Label'] = np.array(train_labels)
X = X_train.values
y = X_train['Label'].values
X = np.delete(X,156,axis=1)


In [124]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)
activityrecogscaler = scaler.fit(X)

In [115]:
import pickle
with open('activityrecogscaler100.pkl', 'wb') as f:
    pickle.dump(activityrecogscaler, f)

In [128]:
b, c = np.unique(y, return_inverse=True)

In [129]:
c

array([5, 5, 5, ..., 3, 3, 3])

In [130]:
#import numpy as np
from sklearn.metrics import matthews_corrcoef
import numpy as np
from sklearn.metrics import confusion_matrix
import numpy as np
from sklearn.metrics import confusion_matrix

def sens(y_true,y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    sensitivity = tp / (tp + fn)
    return sensitivity

 
def spec(y_true,y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    specificity = tn / (tn + fp)
    return specificity   

def f1(y_true,y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    f1_score = 2 * ((sensitivity * specificity) / (sensitivity + specificity))
    return f1_score  

def mcc(y_true,y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    return mcc  
def acc(y_true,y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    return accuracy  

def classification_metrics(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    f1_score = 2 * ((sensitivity * specificity) / (sensitivity + specificity))
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    
    print("Sensitivity: {:.2f}".format(sensitivity))
    print("Specificity: {:.2f}".format(specificity))
    print("F1 score: {:.2f}".format(f1_score))
    print("Accuracy: {:.2f}".format(accuracy))
    print("MCC score: {:.2f}".format(mcc))

In [131]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import ensemble
#Imports - not all were used
from scipy.stats.stats import pearsonr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats
import warnings
from sklearn.metrics import confusion_matrix
import sklearn 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from mpl_toolkits.mplot3d import Axes3D
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from statistics import mean
from sklearn.metrics import balanced_accuracy_score

In [132]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from statistics import mean
from sklearn.metrics import balanced_accuracy_score

import xgboost as xgb


model = xgb.XGBClassifier()
sensi = []
speci = []
f1s = []
accu = []
mccs = []

folds = StratifiedKFold(n_splits = 5)
for train_indeX_normalized, test_indeX_normalized in folds.split(X_normalized,c):
    train1,test1,train2,test2=X_normalized[train_indeX_normalized],X_normalized[test_indeX_normalized],c[train_indeX_normalized],c[test_indeX_normalized]
    model.fit(train1,train2)
    y_pred = model.predict(test1)
    

    print(classification_report(test2,y_pred))



              precision    recall  f1-score   support

           0       0.46      0.40      0.43      1222
           1       0.90      0.80      0.85      3927
           2       1.00      0.99      0.99       687
           3       0.98      0.97      0.98       556
           4       0.25      0.31      0.27      1476
           5       0.79      0.82      0.81      5079

    accuracy                           0.73     12947
   macro avg       0.73      0.72      0.72     12947
weighted avg       0.75      0.73      0.74     12947

              precision    recall  f1-score   support

           0       0.33      0.47      0.39      1222
           1       0.94      0.90      0.92      3928
           2       1.00      0.94      0.97       686
           3       0.93      0.99      0.96       557
           4       0.41      0.47      0.43      1476
           5       0.84      0.75      0.79      5078

    accuracy                           0.75     12947
   macro avg       0.74

In [57]:
z

NameError: name 'z' is not defined

In [68]:
import joblib
joblib.dump(model,'activityrecognition200.joblib')

['activityrecognition200.joblib']

In [41]:
y[12009]

'Downstairs'

In [43]:
c[12009]

0

In [64]:
len(x_list_psd[1000])

4

In [65]:
X_train

,x_mean,y_mean,z_mean,x_std,y_std,z_std,x_aad,y_aad,z_aad,x_min,...,z_skewness_psd,x_kurtosis_psd,y_kurtosis_psd,z_kurtosis_psd,x_energy_psd,y_energy_psd,z_energy_psd,avg_result_accl_psd,sma_psd,Label
0,3.162000,8.838000,-0.355333,5.561302,2.895201,3.089077,4.192800,2.372533,1.929422,-4.59,...,0.782335,-0.811544,-0.982768,-0.893103,81912.561633,839.962501,167.140421,260.064974,149.976014,Walking
1,4.117333,10.388667,-0.112667,6.115939,4.715891,2.744759,5.055822,4.066756,2.238133,-5.13,...,1.069621,-1.378746,-0.863224,-0.741382,24374.862558,12560.384967,6354.780437,222.259246,173.240351,Walking
2,3.739333,10.346667,-1.092667,3.188873,3.142998,2.441615,2.363200,2.504889,1.852444,-1.99,...,0.643871,-1.656969,-1.060669,-1.192161,16195.181815,12733.967186,1227.602446,177.840473,136.214855,Walking
3,3.904667,9.336667,0.306667,6.060706,3.206100,4.152841,4.743644,2.655556,3.227111,-6.66,...,0.494942,-0.678446,-0.667652,-1.021465,221283.577045,5915.695277,5501.535446,389.693270,252.332212,Walking
4,3.063333,10.270000,-0.693333,6.346753,4.537879,4.434250,5.123556,3.716000,3.317333,-7.35,...,1.072850,-0.676067,-1.307711,-0.721178,105531.548992,38269.696719,21947.447243,363.015117,287.844808,Walking
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69043,-1.345333,9.430667,2.468007,0.050049,0.045821,0.038908,0.035200,0.031556,0.030510,-1.50,...,0.491357,-0.821850,-1.162280,-1.229739,0.000042,0.000215,0.000045,0.018980,0.014025,Standing
69044,-1.410667,9.428000,2.451663,0.073072,0.071852,0.053796,0.057778,0.059733,0.043585,-1.53,...,-1.118993,-1.347098,-1.065117,-0.696730,0.002920,0.002827,0.000109,0.086513,0.063055,Standing
69045,-1.340000,9.457333,2.406261,0.063770,0.052848,0.037439,0.053333,0.043556,0.027846,-1.46,...,0.406866,-1.759316,-0.678545,-1.431101,0.001755,0.000330,0.000035,0.050941,0.033170,Standing
69046,-1.386000,9.443333,2.434410,0.064992,0.046857,0.057241,0.059200,0.040889,0.046975,-1.50,...,0.023344,-1.876104,-0.677919,-1.376828,0.000744,0.000666,0.000404,0.047848,0.036347,Standing


In [100]:
for count in range (len(c)):
    if c[count] == 2:
        print(y[count])
        break

Sitting
